# Pragmatic Asset Allocation - Backtest Evaluation

This notebook evaluates the backtest performance of the Pragmatic Asset Allocation strategy against benchmarks and targets.

## Objectives:
- Run comprehensive backtest analysis
- Compare against benchmark portfolios
- Validate performance targets (10.73% annual return, 0.93 Sharpe ratio)
- Analyze risk-adjusted returns and drawdowns
- Test robustness across different market regimes

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load configuration
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully")
print(f"Strategy: {config['strategy']['name']}")
print(f"Backtest period: {config['backtest']['start_date']} to {config['backtest']['end_date']}")
print(f"Target return: {config['strategy']['target_annual_return']:.2%}")
print(f"Target Sharpe: {config['strategy']['target_sharpe_ratio']:.2f}")

## 1. Run Backtest

In [ ]:
# Run the complete backtest
try:
    from backtester import PragmaticAssetAllocationBacktester
    from portfolio_construction import PragmaticAssetAllocationPortfolio
    from signal_generation import PragmaticAssetAllocationSignals
    from data_acquisition import PragmaticAssetAllocationData
    
    # Load data
    data_acq = PragmaticAssetAllocationData()
    all_data = data_acq.load_cached_data()
    
    if all_data:
        # Generate signals
        signal_gen = PragmaticAssetAllocationSignals()
        signals_dict = signal_gen.generate_all_signals(all_data)
        
        # Run portfolio construction
        portfolio = PragmaticAssetAllocationPortfolio()
        portfolio_data = portfolio.run_portfolio_construction(signals_dict, all_data)
        
        # Run backtest
        backtester = PragmaticAssetAllocationBacktester()
        backtest_results = backtester.run_backtest(portfolio_data, all_data)
        
        print("Backtest completed successfully")
        print(f"Backtest results keys: {list(backtest_results.keys())}")
    else:
        print("No data available. Run data acquisition first.")
        backtest_results = {}
        
except ImportError as e:
    print(f"Could not import modules: {e}")
    backtest_results = {}

## 2. Performance Summary

In [ ]:
# Analyze backtest performance summary
if 'performance_summary' in backtest_results:
    perf_summary = backtest_results['performance_summary']
    print("=== PERFORMANCE SUMMARY ===\n")
    
    # Strategy performance
    if 'strategy' in perf_summary:
        strategy_perf = perf_summary['strategy']
        print("Pragmatic Asset Allocation Strategy:")
        print(f"  Total Return: {strategy_perf.get('total_return', 0):.2%}")
        print(f"  Annual Return: {strategy_perf.get('annual_return', 0):.2%}")
        print(f"  Annual Volatility: {strategy_perf.get('annual_volatility', 0):.2%}")
        print(f"  Sharpe Ratio: {strategy_perf.get('sharpe_ratio', 0):.2f}")
        print(f"  Max Drawdown: {strategy_perf.get('max_drawdown', 0):.2%}")
        print(f"  Calmar Ratio: {strategy_perf.get('calmar_ratio', 0):.2f}")
        
        # Target validation
        target_return = config['strategy']['target_annual_return']
        target_sharpe = config['strategy']['target_sharpe_ratio']
        
        print("\nTarget Validation:")
        return_diff = strategy_perf.get('annual_return', 0) - target_return
        sharpe_diff = strategy_perf.get('sharpe_ratio', 0) - target_sharpe
        
        if abs(return_diff) < 0.01:  # Within 1%
            print(f"  Annual Return: ✅ MET ({return_diff:+.2%} vs target)")
        else:
            status = "📈 EXCEEDED" if return_diff > 0 else "📉 BELOW"
            print(f"  Annual Return: {status} ({return_diff:+.2%} vs target)")
        
        if abs(sharpe_diff) < 0.1:  # Within 0.1
            print(f"  Sharpe Ratio: ✅ MET ({sharpe_diff:+.2f} vs target)")
        else:
            status = "📈 EXCEEDED" if sharpe_diff > 0 else "📉 BELOW"
            print(f"  Sharpe Ratio: {status} ({sharpe_diff:+.2f} vs target)")
    
    # Benchmark comparison
    if 'benchmarks' in perf_summary:
        benchmarks = perf_summary['benchmarks']
        print("\nBenchmark Comparison:")
        
        for benchmark_name, benchmark_perf in benchmarks.items():
            print(f"\n{benchmark_name}:")
            print(f"  Annual Return: {benchmark_perf.get('annual_return', 0):.2%}")
            print(f"  Annual Volatility: {benchmark_perf.get('annual_volatility', 0):.2%}")
            print(f"  Sharpe Ratio: {benchmark_perf.get('sharpe_ratio', 0):.2f}")
            print(f"  Max Drawdown: {benchmark_perf.get('max_drawdown', 0):.2%}")
            
            # Strategy vs benchmark
            if 'strategy' in perf_summary:
                strategy_return = perf_summary['strategy'].get('annual_return', 0)
                benchmark_return = benchmark_perf.get('annual_return', 0)
                excess_return = strategy_return - benchmark_return
                
                strategy_sharpe = perf_summary['strategy'].get('sharpe_ratio', 0)
                benchmark_sharpe = benchmark_perf.get('sharpe_ratio', 0)
                excess_sharpe = strategy_sharpe - benchmark_sharpe
                
                print(f"  Excess Return: {excess_return:+.2%}")
                print(f"  Excess Sharpe: {excess_sharpe:+.2f}")
    
else:
    print("Performance summary not available")

## 3. Returns Analysis

In [ ]:
# Analyze returns distribution and time series
if 'returns' in backtest_results:
    returns_data = backtest_results['returns']
    print("=== RETURNS ANALYSIS ===\n")
    
    # Returns summary
    if 'strategy' in returns_data:
        strategy_returns = returns_data['strategy']
        print(f"Strategy returns: {len(strategy_returns)} observations")
        print(f"Date range: {strategy_returns.index.min()} to {strategy_returns.index.max()}")
        
        # Returns statistics
        print("\nStrategy Returns Statistics:")
        print(f"  Mean daily return: {strategy_returns.mean():.3%}")
        print(f"  Daily volatility: {strategy_returns.std():.3%}")
        print(f"  Skewness: {strategy_returns.skew():.2f}")
        print(f"  Kurtosis: {strategy_returns.kurtosis():.2f}")
        print(f"  Best day: {strategy_returns.max():.2%}")
        print(f"  Worst day: {strategy_returns.min():.2%}")
        
        # Returns distribution
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Daily returns histogram
        strategy_returns.hist(bins=50, ax=axes[0,0], alpha=0.7)
        axes[0,0].axvline(strategy_returns.mean(), color='red', linestyle='--', label='Mean')
        axes[0,0].set_title('Daily Returns Distribution')
        axes[0,0].set_xlabel('Daily Return')
        axes[0,0].set_ylabel('Frequency')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        
        # Cumulative returns
        cumulative_returns = (1 + strategy_returns).cumprod()
        cumulative_returns.plot(ax=axes[0,1], linewidth=2)
        axes[0,1].set_title('Cumulative Returns')
        axes[0,1].set_ylabel('Growth of $1')
        axes[0,1].grid(True, alpha=0.3)
        
        # Rolling volatility
        rolling_vol = strategy_returns.rolling(252).std() * np.sqrt(252)
        rolling_vol.plot(ax=axes[1,0], linewidth=1, color='orange')
        axes[1,0].set_title('Rolling 1Y Volatility')
        axes[1,0].set_ylabel('Annualized Volatility')
        axes[1,0].grid(True, alpha=0.3)
        
        # Rolling Sharpe
        risk_free_rate = config['backtest']['risk_free_rate']
        rolling_sharpe = (strategy_returns.rolling(252).mean() * 252 - risk_free_rate) / (strategy_returns.rolling(252).std() * np.sqrt(252))
        rolling_sharpe.plot(ax=axes[1,1], linewidth=1, color='green')
        axes[1,1].axhline(y=target_sharpe, color='red', linestyle='--', alpha=0.7, label='Target Sharpe')
        axes[1,1].set_title('Rolling 1Y Sharpe Ratio')
        axes[1,1].set_ylabel('Sharpe Ratio')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Benchmark comparison
        if 'benchmarks' in returns_data:
            benchmarks = returns_data['benchmarks']
            
            plt.figure(figsize=(15, 8))
            
            # Plot strategy cumulative returns
            (1 + strategy_returns).cumprod().plot(linewidth=3, label='Pragmatic AA', color='blue')
            
            # Plot benchmark cumulative returns
            colors = ['red', 'green', 'orange', 'purple', 'brown']
            for i, (benchmark_name, benchmark_returns) in enumerate(benchmarks.items()):
                if not benchmark_returns.empty:
                    (1 + benchmark_returns).cumprod().plot(linewidth=2, label=benchmark_name, 
                                                           color=colors[i % len(colors)], alpha=0.8)
            
            plt.title('Strategy vs Benchmarks - Cumulative Returns')
            plt.ylabel('Growth of $1')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
    
else:
    print("Returns data not available")

## 4. Risk Analysis

In [ ]:
# Analyze risk metrics and drawdowns
if 'risk_metrics' in backtest_results:
    risk_metrics = backtest_results['risk_metrics']
    print("=== RISK ANALYSIS ===\n")
    
    # Strategy risk metrics
    if 'strategy' in risk_metrics:
        strategy_risk = risk_metrics['strategy']
        print("Strategy Risk Metrics:")
        print(f"  Value at Risk (95%): {strategy_risk.get('var_95', 0):.2%}")
        print(f"  Expected Shortfall (95%): {strategy_risk.get('expected_shortfall_95', 0):.2%}")
        print(f"  Maximum Drawdown: {strategy_risk.get('max_drawdown', 0):.2%}")
        print(f"  Longest Drawdown: {strategy_risk.get('longest_drawdown_days', 0)} days")
        print(f"  Recovery Time: {strategy_risk.get('recovery_time_days', 0)} days")
        
        # Drawdown analysis
        if 'drawdowns' in strategy_risk:
            drawdowns = strategy_risk['drawdowns']
            
            print("\nDrawdown Analysis:")
            print(f"  Total drawdown periods: {len(drawdowns)}")
            print(f"  Average drawdown: {drawdowns['drawdown'].mean():.2%}")
            print(f"  Average recovery time: {drawdowns['recovery_days'].mean():.0f} days")
            
            # Plot drawdowns
            plt.figure(figsize=(15, 10))
            
            plt.subplot(2, 1, 1)
            drawdowns['drawdown'].plot(kind='bar', alpha=0.7)
            plt.title('Drawdown Magnitude by Period')
            plt.ylabel('Drawdown %')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            
            plt.subplot(2, 1, 2)
            drawdowns['recovery_days'].plot(kind='bar', alpha=0.7, color='orange')
            plt.title('Recovery Time by Drawdown Period')
            plt.ylabel('Recovery Days')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # Underwater chart
        if 'underwater' in strategy_risk:
            underwater = strategy_risk['underwater']
            
            plt.figure(figsize=(15, 6))
            underwater.plot(linewidth=1, color='red')
            plt.fill_between(underwater.index, underwater, 0, color='red', alpha=0.3)
            plt.title('Underwater Chart - Strategy Drawdowns')
            plt.ylabel('Drawdown %')
            plt.grid(True, alpha=0.3)
            plt.show()
    
    # Benchmark risk comparison
    if 'benchmarks' in risk_metrics:
        benchmarks_risk = risk_metrics['benchmarks']
        print("\nBenchmark Risk Comparison:")
        
        comparison_data = []
        for benchmark_name, benchmark_risk in benchmarks_risk.items():
            comparison_data.append({
                'Portfolio': benchmark_name,
                'Max Drawdown': benchmark_risk.get('max_drawdown', 0),
                'VaR 95%': benchmark_risk.get('var_95', 0),
                'Recovery Days': benchmark_risk.get('recovery_time_days', 0)
            })
        
        if 'strategy' in risk_metrics:
            strategy_risk = risk_metrics['strategy']
            comparison_data.insert(0, {
                'Portfolio': 'Pragmatic AA',
                'Max Drawdown': strategy_risk.get('max_drawdown', 0),
                'VaR 95%': strategy_risk.get('var_95', 0),
                'Recovery Days': strategy_risk.get('recovery_time_days', 0)
            })
        
        comparison_df = pd.DataFrame(comparison_data)
        print(comparison_df.to_string(index=False, float_format='%.2%'))
    
else:
    print("Risk metrics not available")

## 5. Regime Analysis

In [ ]:
# Analyze performance across different market regimes
if 'regime_analysis' in backtest_results:
    regime_analysis = backtest_results['regime_analysis']
    print("=== REGIME ANALYSIS ===\n")
    
    # Market regime performance
    if 'market_regimes' in regime_analysis:
        market_regimes = regime_analysis['market_regimes']
        print("Performance by Market Regime:")
        
        for regime, perf in market_regimes.items():
            print(f"\n{regime}:")
            print(f"  Annual Return: {perf.get('annual_return', 0):.2%}")
            print(f"  Annual Volatility: {perf.get('annual_volatility', 0):.2%}")
            print(f"  Sharpe Ratio: {perf.get('sharpe_ratio', 0):.2f}")
            print(f"  Max Drawdown: {perf.get('max_drawdown', 0):.2%}")
            print(f"  Observation Days: {perf.get('days', 0)}")
    
    # Signal regime performance
    if 'signal_regimes' in regime_analysis:
        signal_regimes = regime_analysis['signal_regimes']
        print("\nPerformance by Signal Regime:")
        
        for regime, perf in signal_regimes.items():
            print(f"\n{regime}:")
            print(f"  Annual Return: {perf.get('annual_return', 0):.2%}")
            print(f"  Annual Volatility: {perf.get('annual_volatility', 0):.2%}")
            print(f"  Sharpe Ratio: {perf.get('sharpe_ratio', 0):.2f}")
            print(f"  Observation Days: {perf.get('days', 0)}")
    
    # Economic regime performance
    if 'economic_regimes' in regime_analysis:
        economic_regimes = regime_analysis['economic_regimes']
        print("\nPerformance by Economic Regime:")
        
        for regime, perf in economic_regimes.items():
            print(f"\n{regime}:")
            print(f"  Annual Return: {perf.get('annual_return', 0):.2%}")
            print(f"  Annual Volatility: {perf.get('annual_volatility', 0):.2%}")
            print(f"  Sharpe Ratio: {perf.get('sharpe_ratio', 0):.2f}")
            print(f"  Observation Days: {perf.get('days', 0)}")
    
    # Visualize regime performance
    if market_regimes:
        # Create regime performance comparison
        regime_data = []
        for regime, perf in market_regimes.items():
            regime_data.append({
                'Regime': regime,
                'Return': perf.get('annual_return', 0),
                'Volatility': perf.get('annual_volatility', 0),
                'Sharpe': perf.get('sharpe_ratio', 0),
                'Max DD': perf.get('max_drawdown', 0)
            })
        
        regime_df = pd.DataFrame(regime_data)
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Returns by regime
        regime_df.set_index('Regime')['Return'].plot(kind='bar', ax=axes[0,0], color='skyblue')
        axes[0,0].set_title('Annual Returns by Market Regime')
        axes[0,0].set_ylabel('Annual Return %')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].grid(True, alpha=0.3)
        
        # Volatility by regime
        regime_df.set_index('Regime')['Volatility'].plot(kind='bar', ax=axes[0,1], color='orange')
        axes[0,1].set_title('Annual Volatility by Market Regime')
        axes[0,1].set_ylabel('Annual Volatility %')
        axes[0,1].tick_params(axis='x', rotation=45)
        axes[0,1].grid(True, alpha=0.3)
        
        # Sharpe by regime
        regime_df.set_index('Regime')['Sharpe'].plot(kind='bar', ax=axes[1,0], color='green')
        axes[1,0].set_title('Sharpe Ratio by Market Regime')
        axes[1,0].set_ylabel('Sharpe Ratio')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].grid(True, alpha=0.3)
        
        # Max DD by regime
        regime_df.set_index('Regime')['Max DD'].plot(kind='bar', ax=axes[1,1], color='red')
        axes[1,1].set_title('Max Drawdown by Market Regime')
        axes[1,1].set_ylabel('Max Drawdown %')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
else:
    print("Regime analysis not available")

## 6. Attribution Analysis

In [ ]:
# Analyze performance attribution
if 'attribution' in backtest_results:
    attribution = backtest_results['attribution']
    print("=== ATTRIBUTION ANALYSIS ===\n")
    
    # Asset contribution
    if 'asset_contribution' in attribution:
        asset_contrib = attribution['asset_contribution']
        print("Asset Contribution to Returns:")
        
        for asset, contrib in asset_contrib.items():
            pct_contrib = contrib / sum(asset_contrib.values()) * 100
            print(f"  {asset}: {contrib:.2%} ({pct_contrib:.1f}%)")
        
        # Visualize asset contribution
        plt.figure(figsize=(12, 8))
        
        plt.subplot(1, 2, 1)
        pd.Series(asset_contrib).plot(kind='bar', color='skyblue')
        plt.title('Asset Contribution to Total Returns')
        plt.ylabel('Contribution ($)')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        (pd.Series(asset_contrib) / sum(asset_contrib.values())).plot(kind='pie', autopct='%1.1f%%')
        plt.title('Asset Contribution Percentage')
        plt.ylabel('')
        
        plt.tight_layout()
        plt.show()
    
    # Signal contribution
    if 'signal_contribution' in attribution:
        signal_contrib = attribution['signal_contribution']
        print("\nSignal Contribution to Returns:")
        
        for signal, contrib in signal_contrib.items():
            pct_contrib = contrib / sum(signal_contrib.values()) * 100
            print(f"  {signal}: {contrib:.2%} ({pct_contrib:.1f}%)")
        
        # Visualize signal contribution
        plt.figure(figsize=(10, 6))
        pd.Series(signal_contrib).plot(kind='barh', color='lightgreen')
        plt.title('Signal Contribution to Returns')
        plt.xlabel('Contribution ($)')
        plt.grid(True, alpha=0.3)
        plt.show()
    
    # Timing contribution
    if 'timing_contribution' in attribution:
        timing_contrib = attribution['timing_contribution']
        print("\nTiming Contribution:")
        print(f"  Market timing: {timing_contrib.get('market_timing', 0):.2%}")
        print(f"  Signal timing: {timing_contrib.get('signal_timing', 0):.2%}")
        print(f"  Rebalancing timing: {timing_contrib.get('rebalancing_timing', 0):.2%}")
    
else:
    print("Attribution analysis not available")

## 7. Backtest Validation Summary

In [ ]:
# Generate comprehensive backtest validation summary
print("=== BACKTEST VALIDATION SUMMARY ===\n")

validation_summary = {
    'Metric': [],
    'Target': [],
    'Actual': [],
    'Status': [],
    'Difference': []
}

# Performance targets
if 'performance_summary' in backtest_results and 'strategy' in backtest_results['performance_summary']:
    strategy_perf = backtest_results['performance_summary']['strategy']
    
    # Annual return
    target_return = config['strategy']['target_annual_return']
    actual_return = strategy_perf.get('annual_return', 0)
    return_diff = actual_return - target_return
    
    validation_summary['Metric'].append('Annual Return')
    validation_summary['Target'].append(f"{target_return:.2%}")
    validation_summary['Actual'].append(f"{actual_return:.2%}")
    validation_summary['Difference'].append(f"{return_diff:+.2%}")
    
    if abs(return_diff) <= 0.02:  # Within 2%
        validation_summary['Status'].append('✅ MET')
    elif return_diff > 0:
        validation_summary['Status'].append('📈 EXCEEDED')
    else:
        validation_summary['Status'].append('📉 BELOW')
    
    # Sharpe ratio
    target_sharpe = config['strategy']['target_sharpe_ratio']
    actual_sharpe = strategy_perf.get('sharpe_ratio', 0)
    sharpe_diff = actual_sharpe - target_sharpe
    
    validation_summary['Metric'].append('Sharpe Ratio')
    validation_summary['Target'].append(f"{target_sharpe:.2f}")
    validation_summary['Actual'].append(f"{actual_sharpe:.2f}")
    validation_summary['Difference'].append(f"{sharpe_diff:+.2f}")
    
    if abs(sharpe_diff) <= 0.1:
        validation_summary['Status'].append('✅ MET')
    elif sharpe_diff > 0:
        validation_summary['Status'].append('📈 EXCEEDED')
    else:
        validation_summary['Status'].append('📉 BELOW')
    
    # Risk metrics
    max_dd = strategy_perf.get('max_drawdown', 0)
    validation_summary['Metric'].append('Max Drawdown')
    validation_summary['Target'].append('< 20%')
    validation_summary['Actual'].append(f"{max_dd:.1%}")
    validation_summary['Difference'].append('N/A')
    
    if max_dd <= 0.20:
        validation_summary['Status'].append('✅ ACCEPTABLE')
    else:
        validation_summary['Status'].append('⚠️ HIGH')
    
    # Volatility
    volatility = strategy_perf.get('annual_volatility', 0)
    validation_summary['Metric'].append('Annual Volatility')
    validation_summary['Target'].append('< 15%')
    validation_summary['Actual'].append(f"{volatility:.1%}")
    validation_summary['Difference'].append('N/A')
    
    if volatility <= 0.15:
        validation_summary['Status'].append('✅ ACCEPTABLE')
    else:
        validation_summary['Status'].append('⚠️ HIGH')

# Display validation summary
if validation_summary['Metric']:
    validation_df = pd.DataFrame(validation_summary)
    print(validation_df.to_string(index=False))
    
    print("\nValidation Summary:")
    met = validation_summary['Status'].count('✅ MET') + validation_summary['Status'].count('✅ ACCEPTABLE')
    exceeded = validation_summary['Status'].count('📈 EXCEEDED')
    below = validation_summary['Status'].count('📉 BELOW')
    warnings = validation_summary['Status'].count('⚠️ HIGH')
    total = len(validation_summary['Status'])
    
    print(f"Targets Met: {met}/{total} ({met/total:.0%})")
    print(f"Targets Exceeded: {exceeded}/{total}")
    print(f"Targets Below: {below}/{total}")
    print(f"Risk Warnings: {warnings}/{total}")
    
    if met >= total * 0.75 and below == 0:
        print("\n🎉 BACKTEST VALIDATION SUCCESSFUL")
        print("Strategy meets or exceeds performance targets!")
    elif exceeded >= 1:
        print("\n📈 BACKTEST EXCEEDED EXPECTATIONS")
        print("Strategy outperformed targets - consider position sizing!")
    else:
        print("\n⚠️ BACKTEST VALIDATION ISSUES")
        print("Strategy underperformed - review implementation and parameters")
else:
    print("No validation data available")

print("\n=== BACKTEST EVALUATION COMPLETE ===")
print("\nRecommendations:")
print("1. Review validation results against Quant Radio targets")
print("2. Analyze regime-specific performance for robustness")
print("3. Consider out-of-sample testing for forward validation")
print("4. Evaluate transaction costs impact on net returns")
print("5. Assess strategy capacity and liquidity constraints")